In [ ]:
# sudo apt-get update
# sudo apt install chromium-chromedriver
# sudo apt install chromium-bsu

In [ ]:
# from dataclasses import dataclass
# import pandas as pd
# from sqlalchemy.util import dataclass_fields

# from urllib.parse import urljoin
# import requests
# from bs4 import BeautifulSoup
# import re
# from IPython.display import display, HTML


In [ ]:
import logging

import sqlite3
from fastcore.basics import patch_to

# download_url
import time
import selenium




In [ ]:
class Crawler:

    sqlite_driver_db: str
    sqlit_con: sqlite3.connect

    url_to_visit_ls: list[str]

    url_visited_ls: list[str] = []
    article_ls : list[str] = []

    def __init__(self,
                 sqlite_driver_db: str,
                 url_to_visit_ls: list[str] = [],
                 base_url: str = None):

        self.sqlite_driver_db = sqlite_driver_db
        self.sqlit_con = sqlite3.connect(self.sqlite_driver_db)
        self.base_url = base_url
        self.url_to_visit_ls = url_to_visit_ls


    
    def _add_url_to_visit(self, url ):
      if url not in self.url_visited_ls and url not in self.url_to_visit_ls:
        self.url_to_visit_ls.append(url)


In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

cd = Crawler(sqlite_driver_db = 'kb_scrape', base_url = '')

In [ ]:
@patch_to(Crawler)
def run(self: Crawler, debug_prn: bool = False):
    
    counter = 0
    upload_counter = 0
    num_upload = 20
    offset = 0
    max_upload = 10000

    while self.url_to_visit_ls and counter <= max_upload:
        url = self.url_to_visit_ls.pop(0)
        
        logging.info(f'Crawling: {url}')
        
        try:
            # self.crawl(url, debug_prn)

            pass
            
        except Exception:
            logging.exception(f'Failed to crawl: {url}')

        finally:
            self.url_visited_ls.append(url)

        counter += 1

    #     if counter == 10:
    #     self.update_exist_ls()

    #     if counter % num_upload == 0:
    #     offset = num_upload * upload_counter
    #     df = pd.DataFrame(self.article_ls).iloc[offset:]

    #     print(f"rows uploading {offset} - {counter}")

    #     if upload_counter == 0:
    #         self.insert_into_sql(df, is_reset=True)
    #     else:
    #         self.insert_into_sql(df, is_reset=False)

    #     upload_counter += 1

    # df = pd.DataFrame(self.article_ls).iloc[offset:]

    # self.insert_into_sql(df, is_reset=False)

    print('RUN: done crawling')

    return self

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

url_to_visit_ls = [
    'https://domo-support.domo.com/s/article/360047400753?language=en_US', 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS/sharing-access-to-cards-and-pages?language=en_US']

cd = Crawler(sqlite_driver_db='kb_scrape', base_url='', url_to_visit_ls= url_to_visit_ls)

cd.run()

2023-01-25 02:42:51,076 INFO:Crawling: https://domo-support.domo.com/s/article/360047400753?language=en_US
2023-01-25 02:42:51,077 INFO:Crawling: https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS/sharing-access-to-cards-and-pages?language=en_US


RUN: done crawling


<__main__.Crawler>

In [ ]:
@patch_to(Crawler)
def download_url(self: Crawler, url, debug_prn: bool = False):

  options = webdriver.ChromeOptions()
  options.add_argument("-headless")
  options.add_argument("-no-sandbox")
  options.add_argument("-disable-dev-shm-usage")

  wd = webdriver.Chrome("chromedriver", options=options)

  if debug_prn:
        print(f'downloading data from {url}')

  wd.get(url)
  time.sleep(5)  # sleeps for 5 seconds to give page time to load
  source = wd.page_source

  return source


In [ ]:
import time
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("-headless")
options.add_argument("-no-sandbox")
options.add_argument("-disable-dev-shm-usage")

wd = webdriver.Chrome("chromedriver", options=options)

wd.get(url)


WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 1


In [ ]:
@patch_to(Crawler)
def crawl(self : Crawler, url, debug_prn: bool = False):
    
    if debug_prn:
        print(f"CRAWL:  starting crawl - {url}")

    html = self.download_url(url, debug_prn=debug_prn)

    # article = self.get_article(html, url)

    linked_urls_ls = None
    # linked_urls_ls = article.get('linked_urls_ls')

    if not linked_urls_ls or len(linked_urls_ls) == 0:
        return

    for url in linked_urls_ls:
        self._add_url_to_visit(url)


In [ ]:
# def update_exist_ls(self):
#       """updates crawler's visited_urls based on my database of existing kbs"""
#       cursor = self.sqlit_con.execute('''SELECT url FROM kb''')

#       for row in cursor:
#         self.visited_urls.append(row[0])



#     def get_linked_urls(self, html, debug_prn: bool = False):
#       import urllib.parse as url_parse
#       from bs4 import BeautifulSoup

#       soup = BeautifulSoup(html, 'html.parser')

#       linked_urls = []
#       for link in soup.find_all('a'):
#           path = link.get('href')

#           if not path:
#             continue

#           if path.startswith('/'):
#               path = url_parse.urljoin(self.base_url, path)

#           if path.startswith(self.base_url):
#             linked_urls.append(path)

#       return linked_urls



#     def insert_into_sql(self, article_df, is_reset: bool = False):
#       con = sqlite3.connect(
#           "/content/drive/MyDrive/Colab Notebooks/webscrape_domo/my_db2.db")

#       print(article_df.__dict__)

#       article_df.linked_urls = ", ".join(article_df.linked_urls)

#       # display(HTML(article_df.to_html()))

#       if is_reset:
#         article_df.to_sql("kb", con, if_exists="replace")

#       article_df.to_sql("kb", con, if_exists="append")

#     def get_article(self, html, url):
#       soup = BeautifulSoup(html, 'html.parser')

#       form_labels = soup.find_all(class_=['slds-form-element'])

#       linked_urls_ls = self.get_linked_urls(html)

#       article = {"url": url,
#                  "linked_urls": linked_urls_ls}

#       for form in form_labels:
#         rows = list(form.strings)

#         if len(rows) >= 2:
#           title = rows[0]
#           value_ls = rows[1:]

#           value_tx = " ".join(value_ls)

#           value_clean = re.sub(r'( \n.?)+', r'\r', value_tx)
#           article.update({title: value_clean})

#       if article.get('Article Body'):
#         self.article_ls.append(article)
#       else:
#         self.add_url_to_visit(url)

#       return article





In [ ]:
import platform
platform.platform()

'Linux-5.4.0-1100-azure-x86_64-with-glibc2.31'

In [ ]:
class WebScraper(object):
    def __init__(self, urls):
        self.urls = urls
        # Global Place To Store The Data:
        self.all_data = []
        self.master_dict = {}
        # Run The Scraper:
        asyncio.run(self.main())

    async def fetch(self, session, url):
        try:
            async with session.get(url) as response:
                # 1. Extracting the Text:
                text = await response.text()
                # 2. Extracting the  Tag:
                title_tag = await self.extract_title_tag(text)
                return text, url, title_tag
        except Exception as e:
            print(str(e))

    async def extract_title_tag(self, text):
        try:
            soup = BeautifulSoup(text, 'html.parser')
            return soup.title
        except Exception as e:
            print(str(e))

    async def main(self):
        tasks = []
        headers = {
            "user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
        async with aiohttp.ClientSession(headers=headers) as session:
            for url in self.urls:
                tasks.append(self.fetch(session, url))

            htmls = await asyncio.gather(*tasks)
            self.all_data.extend(htmls)

            # Storing the raw HTML data.
            for html in htmls:
                if html is not None:
                    url = html[1]
                    self.master_dict[url] = {
                        'Raw Html': html[0], 'Title': html[2]}
                else:
                    continue



In [7]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=c6b9a562b9a98a1b33b4e3d2ce11b1f25adfee909086f8b83da864a7d5cee901
  Stored in directory: /home/codespace/.cache/pip/wheels/e4/62/1d/d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pyppeteer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
url = "https://domohelp.domo.com/hc/en-us/articles/360043192634-Configuring-a-DataSync-Folder-in-Workbench-5"

In [9]:
# install playwright package:
# !pip install playwright
# install playwright chrome and firefox browsers
!playwright install chrome firefox


Switching to root user to install dependencies...
++ arch
+ [[ x86_64 == \a\a\r\c\h\6\4 ]]
+ [[ ! -f /etc/os-release ]]
++ bash -c 'source /etc/os-release && echo $ID'
+ ID=ubuntu
+ [[ ubuntu != \u\b\u\n\t\u ]]
+ dpkg --get-selections
+ grep -q '^google-chrome[[:space:]]*install$'
+ apt-get update
Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3065 B]
Get:2 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3171 B]
Get:5 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [171 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:3 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease [24.4 kB]  
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2176 B]
Err:2 https://dl.yarnpkg.com/debian stable InRelease                     

In [13]:
from playwright.sync_api import sync_playwright

with sync_playwright() as pw:
    browser = pw.chromium.launch(headless=False)
    context = browser.new_context(viewport={"width": 1920, "height": 1080})
    page = context.new_page()

    page.goto("https://twitch.tv/directory/game/Art")  # go to url
    # wait for content to load
    page.wait_for_selector("div[data-target=directory-first-item]")

    # parsed = []
    # stream_boxes = page.locator(
    #     "//div[contains(@class,'tw-tower')]/div[@data-target]")
    # for box in stream_boxes.element_handles():
    #     parsed.append({
    #         "title": box.query_selector("h3").inner_text(),
    #         "url": box.query_selector(".tw-link").get_attribute("href"),
    #         "username": box.query_selector(".tw-link").inner_text(),
    #         "viewers": box.query_selector(".tw-media-card-stat").inner_text(),
    #         # tags are not always present:
    #         "tags": box.query_selector(".tw-tag").inner_text() if box.query_selector(".tw-tag") else None,
    #     })
 

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [1]:
pip install requests_html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
  Using cached pyee-8.2.2-py2.py3-none-any.whl (12 kB)
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=51c4cb2d5e04eb92dbddae7ef79f589092f9e699316b37404d6698dba6e68d99
  Stored in directory: /home/codespace/.cache/pip/wheels/03/d9/92/db136347b5bcba7d271a3c042ce8c9c279e0ecd79173bb0a6e
Successfully built parse
  Attempting uninstall: pyee
    Found existing installation: pyee 9.0.4
    Uninstalling pyee-9.0.4:
      Successfully uninstalled pyee-9.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
playwright 1.30.0 requires pyee==9.0.4, but you have pyee 8.2.2 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need 

In [1]:
from requests_html import AsyncHTMLSession

url = 'https://domohelp.domo.com/hc/en-us/articles/360047400753-jupyter-workspaces#11.0.5.'


session = AsyncHTMLSession()
results = await session.get(url)

await results.html.render()


RuntimeError: This event loop is already running